# Space Setup

In [1]:
import sys
import os
import logging
import pandas as pd
import datasets
from datasets import disable_caching; disable_caching()
from pprint import pprint
KEY = '2-OhioT1DM'
WORKSPACE_PATH = os.getcwd().split(KEY)[0]
print(WORKSPACE_PATH); os.chdir(WORKSPACE_PATH)
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='[%(levelname)s:%(asctime)s:(%(filename)s@%(lineno)d %(name)s)]: %(message)s')

SPACE = {
    'DATA_RAW': f'_Data/0-Data_Raw',
    'DATA_RFT': f'_Data/1-Data_RFT',
    'DATA_CASE': f'_Data/2-Data_CASE',
    'DATA_AIDATA': f'_Data/3-Data_AIDATA',
    'DATA_EXTERNAL': f'code/external',
    'DATA_HFDATA': f'_Data/5-Data_HFData',
    'CODE_FN': f'code/pipeline',
    'MODEL_ROOT': f'./_Model',
}
assert os.path.exists(SPACE['CODE_FN']), f'{SPACE["CODE_FN"]} not found'
print(SPACE['CODE_FN'])
sys.path.append(SPACE['CODE_FN'])

# os.environ["CUDA_VISIBLE_DEVICES"]="1"

/home/jluo41/CGMLSM-Project/
code/pipeline


/home/jluo41/miniconda3/envs/nix/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# AI Data

In [2]:
HFDataName = 'OhioT1DM-Split_v0628'
path = os.path.join(SPACE['DATA_HFDATA'], HFDataName)
split_to_dataset = datasets.load_from_disk(path)
remove_unused_columns = True # if using the processed dataset, set to True. 
print(split_to_dataset)
Name_to_Data = {i: {'ds_tfm': split_to_dataset[i]} for i in split_to_dataset}
# Name_to_Data


# DatasetDict({
#     train: Dataset({
#         features: ['PID', 'final_split', 'input_ids', 'hm_ids', 'labels'],
#         num_rows: 37331
#     })
#     valid: Dataset({
#         features: ['PID', 'final_split', 'input_ids', 'hm_ids', 'labels'],
#         num_rows: 4153
#     })
#     test-id: Dataset({
#         features: ['PID', 'final_split', 'input_ids', 'hm_ids', 'labels'],
#         num_rows: 7456
#     })
# })

DatasetDict({
    train: Dataset({
        features: ['PID', 'final_split', 'input_ids', 'hm_ids', 'labels'],
        num_rows: 44275
    })
    valid: Dataset({
        features: ['PID', 'final_split', 'input_ids', 'hm_ids', 'labels'],
        num_rows: 4925
    })
    test-id: Dataset({
        features: ['PID', 'final_split', 'input_ids', 'hm_ids', 'labels'],
        num_rows: 9214
    })
})


In [4]:
import pandas as pd
for Name, Data in Name_to_Data.items():
    df = Data['ds_tfm'].to_pandas()
    df['unique_id'] = Name + '-' + df.index.astype(str)
    print(Name)
    print(df.shape)
    print(df['PID'].nunique())

train
(44275, 6)
12
valid
(4925, 6)
12
test-id
(9214, 6)
11


In [5]:
44275 + 4925 + 9214

58414

In [5]:
CURRENT_INDEX = 289

ds = Name_to_Data['test-id']['ds_tfm']
df = ds.to_pandas()
df['hm_obs'] = df['hm_ids'].apply(lambda x: x[CURRENT_INDEX - 1])
df['hm_obs'] % 1 == 0

0       True
1       True
2       True
3       True
4       True
        ... 
9209    True
9210    True
9211    True
9212    True
9213    True
Name: hm_obs, Length: 9214, dtype: bool

In [7]:

def convert_hfds_to_nixtladf(ds, name, sliding_windows = 12):
    df = ds.to_pandas()
    df['unique_id'] = name + '-' + df.index.astype(str)

    df['hm_obs'] = df['hm_ids'].apply(lambda x: x[CURRENT_INDEX - 1])
    index = df['hm_obs'] % sliding_windows == 0
    df = df[index].reset_index(drop=True)

    
    li = []
    for idx, row in df.iterrows():
        d = {}
        cgm = list(row['input_ids']) + list(row['labels'])
        hm_ids = list(row['hm_ids'])
        assert len(cgm) == len(hm_ids)

        d['y'] = cgm 
        d['hm_ids'] = hm_ids 

        date = pd.to_datetime('2020-01-01')
        start_date = date + pd.Timedelta(minutes=5 * hm_ids[0])
        ds = [start_date + pd.Timedelta(minutes=5 * i) for i in range(len(cgm))]


        # ds = [date + pd.Timedelta(minutes=5 * i) for i in hm_ids]
        d['PID'] = len(cgm) * [row['PID']]
        d['ds'] = ds
        d['unique_id'] = len(cgm) * [row['unique_id']]
        d['stratum'] = len(cgm) * ['T1D']
        d['abs_step'] = range(0, len(cgm))
        df_row = pd.DataFrame(d) 
        li.append(df_row)

    df_nixtla = pd.concat(li).reset_index(drop = True)
    return df_nixtla 


Name = 'test-id'
ds = Name_to_Data[Name]['ds_tfm']
sliding_windows = 1
df = convert_hfds_to_nixtladf(ds, Name, sliding_windows = sliding_windows)
df

,y,hm_ids,PID,ds,unique_id,stratum,abs_step
0,177,28,110000000001,2020-01-01 02:20:00,test-id-0,T1D,0
1,176,29,110000000001,2020-01-01 02:25:00,test-id-0,T1D,1
2,174,30,110000000001,2020-01-01 02:30:00,test-id-0,T1D,2
3,170,31,110000000001,2020-01-01 02:35:00,test-id-0,T1D,3
4,166,32,110000000001,2020-01-01 02:40:00,test-id-0,T1D,4
...,...,...,...,...,...,...,...
2883977,92,118,110000000023,2020-01-02 09:50:00,test-id-9213,T1D,308
2883978,93,119,110000000023,2020-01-02 09:55:00,test-id-9213,T1D,309
2883979,90,120,110000000023,2020-01-02 10:00:00,test-id-9213,T1D,310
2883980,86,121,110000000023,2020-01-02 10:05:00,test-id-9213,T1D,311


In [8]:
df[['PID', 'unique_id']].value_counts().sort_index()

PID           unique_id   
110000000001  test-id-0       313
              test-id-1       313
              test-id-10      313
              test-id-100     313
              test-id-101     313
                             ... 
110000000023  test-id-9209    313
              test-id-9210    313
              test-id-9211    313
              test-id-9212    313
              test-id-9213    313
Name: count, Length: 9214, dtype: int64

In [9]:
Name_to_nixtlaDF = {}

for Name in Name_to_Data:
    ds = Name_to_Data[Name]['ds_tfm']
    print(Name, ds.shape)

    if Name == 'train':
        sliding_windows = 3
    else:
        sliding_windows = 1

    df = convert_hfds_to_nixtladf(ds, Name, sliding_windows = sliding_windows)
    Name_to_nixtlaDF[Name] = df 


NixDataName = 'OhioT1DM-Nixtla-v0628' # new
path = os.path.join(SPACE['DATA_HFDATA'], NixDataName)


train (44275, 5)
valid (4925, 5)
test-id (9214, 5)


In [10]:
if not os.path.exists(path):
    os.makedirs(path)

for Name in Name_to_nixtlaDF:
    df = Name_to_nixtlaDF[Name]
    print(Name, df.shape)
    df.to_parquet(os.path.join(path, f'{Name}.parquet'), index=False)

train (4620819, 7)
valid (1541525, 7)
test-id (2883982, 7)


In [ ]:
# import pandas as pd
# date = pd.to_datetime('2020-01-01') + pd.Timedelta(minutes=5 * (288 + 12))
#         # ds = [date + pd.Timedelta(minutes=5 * i) for i in hm_ids]
# date 